In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
from Managers.GameDirector import GameDirector
from Bots import RandomBot, AlexPastorBot, EdoBot, AdrianHerasBot
import pandas as pd
from tqdm import tqdm
import random

# bots_configs = [
#     [EdoBot.EdoBot, RandomBot.RandomBot, RandomBot.RandomBot, RandomBot.RandomBot],
#     [EdoBot.EdoBot, AlexPastorBot.AlexPastorBot, AlexPastorBot.AlexPastorBot, AlexPastorBot.AlexPastorBot],
#     # [EdoBot.EdoBot, AdrianHerasBot.AdrianHerasBot, AdrianHerasBot.AdrianHerasBot, AdrianHerasBot.AdrianHerasBot],
#     [EdoBot.EdoBot, EdoBot.EdoBot, RandomBot.RandomBot, RandomBot.RandomBot],
#     [EdoBot.EdoBot, EdoBot.EdoBot, AlexPastorBot.AlexPastorBot, AlexPastorBot.AlexPastorBot],
#     # [EdoBot.EdoBot, EdoBot.EdoBot, AdrianHerasBot.AdrianHerasBot, AdrianHerasBot.AdrianHerasBot],
#     [EdoBot.EdoBot, EdoBot.EdoBot, EdoBot.EdoBot, RandomBot.RandomBot],
#     [EdoBot.EdoBot, EdoBot.EdoBot, EdoBot.EdoBot, AlexPastorBot.AlexPastorBot],
#     # [EdoBot.EdoBot, EdoBot.EdoBot, EdoBot.EdoBot, AdrianHerasBot.AdrianHerasBot],
#     ]

bots_configs = [
    [EdoBot.EdoBot, RandomBot.RandomBot, RandomBot.RandomBot, RandomBot.RandomBot],
    [EdoBot.EdoBot, AlexPastorBot.AlexPastorBot, AlexPastorBot.AlexPastorBot, AlexPastorBot.AlexPastorBot],
    # [EdoBot.EdoBot, EdoBot.EdoBot, RandomBot.RandomBot, RandomBot.RandomBot],
    # [EdoBot.EdoBot, EdoBot.EdoBot, EdoBot.EdoBot, RandomBot.RandomBot],
    ]

def shuffle_bots(bots):
    names = [f"{bot.__name__}_{j}" for j, bot in enumerate(bots)]
    temp = list(zip(names, bots))
    random.shuffle(temp)
    res1, res2 = zip(*temp)
    names, bots = list(res1), list(res2)
    return names, bots

def play_single_game(bots):
    names, bots = shuffle_bots(bots)
    game_director = GameDirector(bots = bots)
    game_director.game_start(0)
    last_round = list(game_director.trace_loader.current_trace["game"].values())[-1]
    last_turn = list(last_round.values())[-1]
    points = list(last_turn["end_turn"]["victory_points"].values())
    points = [int(point) for point in points]
    return {name: points[i] for i, name in enumerate(names)}


def play_set_of_games(bots, games_to_play=100):
    return [play_single_game(bots) for i in tqdm(range(games_to_play))]

In [45]:
from joblib import delayed, Parallel

# results_list = Parallel(n_jobs=3)(delayed(play_set_of_games)(bots) for bots in bots_configs)
results_list = [ play_set_of_games(bots) for bots in bots_configs]

for results in results_list:
    df = pd.DataFrame(results)
    max_counts = dict(df.idxmax(axis=1).value_counts())
    bot_wins = {bot: max_counts.get(bot, 0)/len(df.index)*100 for bot in list(df.columns)}

    display(pd.DataFrame(bot_wins, index=["% wins"]).T)








































































































































































































































100%|██████████| 100/100 [00:16<00:00,  6.16it/s]
















































  0%|          | 0/100 [02:21<?, ?it/s]




















































































































































































100%|██████████| 100/100 [00:19<00:00,  5.01it/s]


,% wins
RandomBot_2,31.0
RandomBot_3,22.0
RandomBot_1,23.0
EdoBot_0,24.0


,% wins
AlexPastorBot_3,33.0
AlexPastorBot_1,18.0
AlexPastorBot_2,25.0
EdoBot_0,24.0


In [50]:
from Bots.helpers import *
from Bots.logger import *

clear_log_data()
points = play_single_game([EdoBot.EdoBot, AlexPastorBot.AlexPastorBot, AlexPastorBot.AlexPastorBot, AlexPastorBot.AlexPastorBot])
winer = max(points, key=points.get)
print(points)
print(winer)

{'AlexPastorBot_3': 4, 'EdoBot_0': 3, 'AlexPastorBot_2': 3, 'AlexPastorBot_1': 10}
AlexPastorBot_1


In [51]:
log_data = get_log_df()
display(log_data[["materials", "goal_1", "trade_1", "trade_2", "trade_3" ]].head(50))

,materials,goal_1,trade_1,trade_2,trade_3
0,3🥖 4🪨 0🧱 1🪵 2🧶,🏢 (2🥖3🪨),,,
1,3🥖 1🪨 0🧱 1🪵 2🧶,🛣️ (1🧱1🪵),1🥖 0🪨 0🧱 0🪵 0🧶=> 0🥖 0🪨 1🧱 0🪵 0🧶,,
2,3🥖 1🪨 0🧱 1🪵 2🧶,🛣️ (1🧱1🪵),1🥖 0🪨 0🧱 0🪵 0🧶=> 0🥖 0🪨 1🧱 0🪵 0🧶,,
3,3🥖 1🪨 0🧱 1🪵 2🧶,🛣️ (1🧱1🪵),0🥖 1🪨 0🧱 0🪵 0🧶=> 0🥖 0🪨 1🧱 0🪵 0🧶,,
4,1🥖 0🪨 0🧱 0🪵 2🧶,🛣️ (1🧱1🪵),1🥖 0🪨 0🧱 0🪵 0🧶=> 0🥖 0🪨 0🧱 1🪵 0🧶,,
5,1🥖 0🪨 0🧱 0🪵 2🧶,🛣️ (1🧱1🪵),0🥖 0🪨 0🧱 0🪵 1🧶=> 0🥖 0🪨 1🧱 0🪵 0🧶,,
6,1🥖 0🪨 0🧱 0🪵 2🧶,🛣️ (1🧱1🪵),1🥖 0🪨 0🧱 0🪵 0🧶=> 0🥖 0🪨 1🧱 0🪵 0🧶,,
7,1🥖 0🪨 0🧱 4🪵 2🧶,🛣️ (1🧱1🪵),0🥖 0🪨 0🧱 1🪵 0🧶=> 0🥖 0🪨 1🧱 0🪵 0🧶,,
8,1🥖 0🪨 0🧱 4🪵 2🧶,🛣️ (1🧱1🪵),0🥖 0🪨 0🧱 0🪵 1🧶=> 0🥖 0🪨 1🧱 0🪵 0🧶,,
9,1🥖 0🪨 0🧱 4🪵 2🧶,🛣️ (1🧱1🪵),1🥖 0🪨 0🧱 0🪵 0🧶=> 0🥖 0🪨 1🧱 0🪵 0🧶,,
